# Semi-supervised learning on renders 


In [5]:
# !pip install "tensorflow_hub>=0.6.0"
# !pip install "tensorflow>=2.0.0" uncomment this if tensorflow not found

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import tensorflow as tf


In [36]:
# Process Image
    
IMG_WIDTH, IMG_HEIGHT = 224,224
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  img_resize = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
  return img_resize

def process_path(file_path):
  # label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, img
  # return img, label

In [37]:
list_ds = tf.data.Dataset.list_files(str('../render/render*'))
for f in list_ds.take(5):
  print(f)    

tf.Tensor(b'../render/render6920.png', shape=(), dtype=string)
tf.Tensor(b'../render/render9853.png', shape=(), dtype=string)
tf.Tensor(b'../render/render923.png', shape=(), dtype=string)
tf.Tensor(b'../render/render776.png', shape=(), dtype=string)
tf.Tensor(b'../render/render9039.png', shape=(), dtype=string)


In [3]:
import keras.backend as K

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score
    

NameError: name 'md' is not defined

In [39]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
image_count = 10394
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
for item, mirror_item in labeled_ds.take(1):
    ImgShape = item.shape
# df = pd.DataFrame(labeled_ds)
# df.info()

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()
  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [45]:
def GetModel(input_shape):
    input_img = Input(shape=(224, 224, 3))

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(2, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    # Output Shape: 4x4x8
    x = Conv2D(2, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    # Output Shape: 28x28x1
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer="adadelta", loss="mse", metrics=['acc']) # may add f1_score later
    autoencoder.summary()
    return autoencoder

Dataset_train = prepare_for_training(labeled_ds)

In [46]:
autoencoder = GetModel(input_shape=ImgShape)
X_train

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 112, 112, 8)       1160      
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 56, 56, 8)         0         
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 56, 56, 4)         292       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 28, 28, 4)         0  

<PrefetchDataset shapes: (None, 224, 224, 3), types: tf.float32>

In [47]:

# Use unlabeled data to train the Encoder
# autoencoder.fit(X_train, X_train, steps_per_epoch=16, epochs=10, verbose=1,
#                shuffle=True)

# Data_train = tf.data.Dataset.from_tensor_slices((X_train, X_train))
batch_stats_callback = CollectBatchStats()
history = autoencoder.fit_generator(Dataset_train, epochs=100,
                              steps_per_epoch=STEPS_PER_EPOCH,
                              callbacks = [batch_stats_callback])

autoencoder.save('./saved_model/autoencoder.h5')

Train for 325.0 steps
Epoch 1/100
325/325 [==============================] - 244s 752ms/step - loss: 0.1861 - acc: 0.1672
Epoch 3/100
325/325 [==============================] - 247s 760ms/step - loss: 0.1856 - acc: 0.1443
Epoch 4/100
325/325 [==============================] - 246s 758ms/step - loss: 0.1851 - acc: 0.1171
Epoch 5/100
325/325 [==============================] - 246s 758ms/step - loss: 0.1847 - acc: 0.1229
Epoch 6/100
325/325 [==============================] - 246s 758ms/step - loss: 0.1839 - acc: 0.1826
Epoch 7/100
 72/325 [=====>........................] - ETA: 3:15 - loss: 0.1837 - acc: 0.1302

KeyError: 'loss'